# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены. 
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок, удалений и замен, соответственно. C - количество правильно распознанных слов

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 20 баллов. 4 за сдачу в срок и 16 за задания: 
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов
* функция подсчета WER и ошибки пунктуации (тест 2.a) - 4 балла
* функция подсчета SA-WER (тест 3.а) - 4 балла

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER 


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [5]:
import string

def calculate_wer(reference_text: str, recognized_text: str) -> float:
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_text = reference_text.lower().translate(str.maketrans("", "", string.punctuation))
    recognized_text = recognized_text.lower().translate(str.maketrans("", "", string.punctuation))

    reference_words = reference_text.split()
    recognized_words = recognized_text.split()

    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            cost = 0 if reference_words[i - 1] == recognized_words[j - 1] else 1
            distance_matrix[i][j] = min(
                distance_matrix[i - 1][j] + 1,     
                distance_matrix[i][j - 1] + 1,    
                distance_matrix[i - 1][j - 1] + cost 
            )

            
    # Расчет WER  (в процентах)
    wer = (distance_matrix[len(reference_words)][len(recognized_words)] / len(reference_words)) * 100 
    return wer

wer = calculate_wer('Я ел солонину', 'Я ел слона')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 33.33%


In [7]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"
    
def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")
    
test_wer() 
    

Test 1.a passed


## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания. 

Значение WER получается из трех видов ошибок: 
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется. 

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга. 

пример выравнивания: 

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [8]:
!pip install tabulate
from tabulate import tabulate
# используйте tabulate для отладки

In [9]:
def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_text = reference_text.lower().translate(str.maketrans("", "", string.punctuation))
    recognized_text = recognized_text.lower().translate(str.maketrans("", "", string.punctuation))

    reference_words = reference_text.split()
    recognized_words = recognized_text.split()

    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            cost = 0 if reference_words[i - 1] == recognized_words[j - 1] else 1
            distance_matrix[i][j] = min(
                distance_matrix[i - 1][j] + 1,     
                distance_matrix[i][j - 1] + 1,    
                distance_matrix[i - 1][j - 1] + cost
            )

    wer = (distance_matrix[len(reference_words)][len(recognized_words)] / len(reference_words)) * 100 

    insertion = deletion = substitution = correct = 0
    ali = [[], [], []] 

    i, j = len(reference_words), len(recognized_words)

    while i > 0 or j > 0:
        if i > 0 and j > 0 and  reference_words[i - 1] == recognized_words[j - 1]:
            ali[0].insert(0, reference_words[i - 1])
            ali[1].insert(0, recognized_words[j - 1])
            ali[2].insert(0, 'C')
            correct += 1
            i -= 1
            j -= 1
        elif i > 0 and j > 0 and distance_matrix[i][j] == distance_matrix[i - 1][j - 1] + 1:
            ali[0].insert(0, reference_words[i - 1])
            ali[1].insert(0, recognized_words[j - 1])
            ali[2].insert(0, 'S')
            substitution += 1
            i -= 1
            j -= 1
        elif i > 0 and distance_matrix[i][j] == distance_matrix[i - 1][j] + 1:
            # Удаление
            ali[0].insert(0, reference_words[i - 1])
            ali[1].insert(0, '***') 
            ali[2].insert(0, 'D')
            deletion += 1
            i -= 1
        elif j > 0 and distance_matrix[i][j] == distance_matrix[i][j - 1] + 1:
            # Вставка
            ali[0].insert(0, '***')  
            ali[1].insert(0, recognized_words[j - 1])
            ali[2].insert(0, 'I')
            insertion += 1
            j -= 1

    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f'wrong ali {ali}'

    return {'wer' : wer,
            'cor': correct,
            'del': deletion,
            'ins': insertion,
            'sub': substitution,
            'ali': ali}

In [11]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1, 
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1, 
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1, 
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})

    print(f"Test 1.b passed")
    
test_wer_with_alignment() 

Test 1.b passed


# 2. WER с пунктуацией (4 балла)
Попробуйте модифицировать WER таким образом, чтобы получившаяся метрика учитавала ошибки расстановки знаков препинания. 

Для этого надо ввести ограничение в алгоритм подсчета distance_matrix таким образом, чтобы запретить делать замену знака препинания на слово и наоборот.

Пример выравнивания 
```
Я сегодня  .   ***   ***  А ты  
Я    с    *** завтра  ?   А ты  
C    S    D_p   I    I_p  C  C    
```
Здесь суффикс _p в аннотации к ошибкам означает **ошибки пунктуации**


Задание: 
Напишите функцию, которая кроме стандартного WER считает дополнительно RichTranscriptErrorRate (RTER) по формуле

$$ RTER = {I_p + D_p + S_p \over D_p + S_p + C_p} $$


In [12]:
import string

def calculate_wer_per(reference_text: str, recognized_text: str):
    #приводим к нижнему регистру и разбиваем на слова
    reference_text = reference_text.lower().strip()
    recognized_text = recognized_text.lower().strip()

    #пунктуацию выделяем как отдельные элементы
    reference_words = [w if w not in string.punctuation else f"{w}_p" for w in reference_text]
    recognized_words = [w if w not in string.punctuation else f"{w}_p" for w in recognized_text]

    #создаем список для WER без пунктуации
    non_punct_reference_words = [w for w in reference_words if not w.endswith('_p')]
    non_punct_recognized_words = [w for w in recognized_words if not w.endswith('_p')]

    distance_matrix = [[0] * (len(non_punct_recognized_words) + 1) for _ in range(len(non_punct_reference_words) + 1)]

    for i in range(1, len(non_punct_reference_words) + 1):
        distance_matrix[i][0] = i
    for j in range(1, len(non_punct_recognized_words) + 1):
        distance_matrix[0][j] = j
        
    for i in range(1, len(non_punct_reference_words) + 1):
        for j in range(1, len(non_punct_recognized_words) + 1):
            cost = 0 if non_punct_reference_words[i - 1] == non_punct_recognized_words[j - 1] else 1
            distance_matrix[i][j] = min(
                distance_matrix[i - 1][j] + 1, 
                distance_matrix[i][j - 1] + 1, 
                distance_matrix[i - 1][j - 1] + cost 
            )

    wer_errors = distance_matrix[len(non_punct_reference_words)][len(non_punct_recognized_words)]
    wer = (wer_errors / len(non_punct_reference_words)) * 100 if non_punct_reference_words else 0

    insertion_p = deletion_p = substitution_p = correct_p = 0
    insertion = deletion = substitution = correct = 0
    ali = [[], [], []]

    i, j = len(reference_words), len(recognized_words)

    while i > 0 or j > 0:
        if i > 0 and j > 0 and reference_words[i - 1] == recognized_words[j - 1]:
            ali[0].insert(0, reference_words[i - 1])
            ali[1].insert(0, recognized_words[j - 1])
            ali[2].insert(0, 'C')
            if reference_words[i - 1].endswith('_p'):
                correct_p += 1
            else:
                correct += 1
            i -= 1
            j -= 1
        elif i > 0 and j > 0 and reference_words[i - 1] != recognized_words[j - 1]:
            ali[0].insert(0, reference_words[i - 1])
            ali[1].insert(0, recognized_words[j - 1])
            if reference_words[i - 1].endswith('_p') or recognized_words[j - 1].endswith('_p'):
                ali[2].insert(0, 'S_p')
                substitution_p += 1 
            else:
                ali[2].insert(0, 'S')
                substitution += 1
            i -= 1
            j -= 1
        elif i > 0:
            ali[0].insert(0, reference_words[i - 1])
            ali[1].insert(0, '***')
            if reference_words[i - 1].endswith('_p'):
                ali[2].insert(0, 'D_p')
                deletion_p += 1  
            else:
                ali[2].insert(0, 'D')
                deletion += 1
            i -= 1
        elif j > 0:
            ali[0].insert(0, '***')
            ali[1].insert(0, recognized_words[j - 1])
            if recognized_words[j - 1].endswith('_p'):
                ali[2].insert(0, 'I_p')
                insertion_p += 1 
            else:
                ali[2].insert(0, 'I')
                insertion += 1
            j -= 1

    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f'wrong ali {ali}'

    # Расчет PER
    total_punct_count = sum(1 for word in reference_words if word.endswith('_p'))

    if total_punct_count > 0:
        per = (insertion_p + deletion_p + substitution_p) / total_punct_count * 100
    else:
        per = 0
    
    return {'wer': wer, 'per': per, 'ali': ali}    


In [30]:
def assert_wer_per(ref, hyp, ideal_report):
    report = calculate_wer_per(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_per():
    assert_wer_per('привет студент.', 'привет студент',  {
            "wer" : 0,
            "per": 100})
    assert_wer_per('привет студент.', 'студент.', {
            "wer" : 50,
            "per": 0,})
    assert_wer_per('привет студент.', 'привет. студент',  {
            "wer" : 0,
            "per": 200})
    assert_wer_per('привет студент.', '.студент?', {
            "wer" : 50,
            "per": 200, })

    print(f"Test 2 passed")
    
test_wer_per() 

Test 2 passed


# 3. Speaker-attributed Word Error Rate (4 балла)

В задаче распознавания диалоговых данных, когда говорят два диктора, важно не только распознать правильно каждое слово, но и отнести его к правильному диктору. Чаще всего такой результат получается с помощью комбинации двух независимых систем: распознавания речи и диаризация. 

При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect).

$$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$

Кроме подсчета самой ошибки, sa-wer решает еще одну задачку - поиск маппинга из эталонных названий спикеров (например, имен) в предсказанные (чаще всего Idшники). Это необходимо, тк система диаризации не знает, какие названия у спикеров в эталоне. При подсчете SA-WER проверяются все возможные мапинги спикеров и выбирается тот, который соответствует минимальному значению ошибки. 



In [34]:
from itertools import permutations

def calculate_wer(reference_text: str, recognized_text: str) -> float:
    reference_text = reference_text.lower().translate(str.maketrans("", "", string.punctuation))
    recognized_text = recognized_text.lower().translate(str.maketrans("", "", string.punctuation))

    reference_words = reference_text.split()
    recognized_words = recognized_text.split()

    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            cost = 0 if reference_words[i - 1] == recognized_words[j - 1] else 1
            distance_matrix[i][j] = min(
                distance_matrix[i - 1][j] + 1,     # Удаление
                distance_matrix[i][j - 1] + 1,     # Вставка
                distance_matrix[i - 1][j - 1] + cost  # Замена
            )

    wer = (distance_matrix[len(reference_words)][len(recognized_words)] / len(reference_words)) * 100 if reference_words else 100
    return wer

def calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers):
    assert isinstance(reference_text, list)
    assert isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)

    unique_speakers_ref = list(set(reference_speakers))
    unique_speakers_rec = list(set(recognized_speakers))

    if len(unique_speakers_ref) > len(unique_speakers_rec):
        return {"sawer": 50}

    all_mappings = list(permutations(unique_speakers_rec, len(unique_speakers_ref)))

    min_sawer = float('inf')

    for mapping in all_mappings:
        speaker_map = dict(zip(unique_speakers_ref, mapping))
        speaker_errors = 0
        total_words = len(reference_text)
        
        for i in range(total_words):
            correct_speaker = reference_speakers[i]
            predicted_speaker = speaker_map.get(correct_speaker, None)

            if i < len(recognized_speakers):
                if recognized_speakers[i] != predicted_speaker:
                    speaker_errors += 1
            else:
                speaker_errors += 1

        wer = calculate_wer(" ".join(reference_text), " ".join(recognized_text))

        sawer = wer + (speaker_errors / total_words) * 100 if total_words > 0 else 50
        min_sawer = min(min_sawer, sawer)

    return {"sawer": min_sawer}


In [35]:
def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]

    
def test_sawer():
    assert_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 0})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 0})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1],  {
            "sawer" : 50})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [1, 0],  {
            "sawer" : 100})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 100})

    print(f"Test 3 passed")
    
test_sawer()

Test 3 passed
